<a href="https://colab.research.google.com/github/Computer-CGuy/MusgraveTextureNeuralRenderer/blob/models/SKIN_CANCER_%7C_NeuralRenderer_0.13_L1Loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch  
import matplotlib.pyplot as plt  
import numpy as np  
import torch.nn.functional as func
import torch.nn.functional as F
import PIL.ImageOps  
from torch import nn  
from torchvision import datasets,transforms   
import requests  
from PIL import Image  
import torchvision.models as models
import os
import json
from torch.utils.data import Dataset, DataLoader, Subset
import cv2
from google.colab.patches import cv2_imshow
import torchvision
import pickle

In [2]:
net = models.resnext50_32x4d(pretrained=True).cuda()

Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth


In [3]:
net

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

In [4]:
net.fc = nn.Sequential(
          nn.Linear(2048,1024),
          nn.LeakyReLU(),
          nn.Linear(1024,4),
          nn.LeakyReLU()
        ).cuda()

In [5]:
from PIL import Image

In [8]:
!ls "/content/drive/My Drive/MusgraveTextureNeuralRenderer/GeneratorDictionaries" | wc -l

10251


In [7]:
# with open("/content/drive/My Drive/MusgraveTextureNeuralRenderer/GeneratorDictionaries/out.pkl","rb") as f:
#     l = (pickle.load(f))

In [11]:
!ls "/content/drive/My Drive/MusgraveTextureNeuralRenderer/GeneratorDictionaries/" | grep ".*pkl"

out.pkl


In [10]:
!ls "/content/drive/My Drive/*/" | wc -l

ls: cannot access '/content/drive/My Drive/*/': No such file or directory
0


In [9]:
class MusgraveDataset(Dataset):
    def __init__(self,transforms):
        with open("/content/drive/My Drive/MusgraveTextureNeuralRenderer/GeneratorDictionaries/out.pkl","rb") as f:
            self.l = (pickle.load(f))
        self.transforms = transforms
    def __getitem__(self, index):
        # print(index)
        l = self.l
        file_name = l[index]["name"]
        file_name = "/content/drive/My Drive/MusgraveTextureNeuralRenderer/GeneratorDictionaries/"+file_name
        img = Image.open(file_name)
        # img = cv2.imread(file_name)
        # img = np.transpose(img,(2,0,1))
        # transforms.Compose([
            
        # ])
        return ((self.transforms(img),torch.Tensor(l[index]["values"])))
    
    def __len__(self):
        return len(self.l)

In [ ]:
# !ls "./drive/My Drive/MusgraveTextureNeuralRenderer/Generator/*.pkl"

In [ ]:
# import pickle
# with open("./drive/My Drive/MusgraveTextureNeuralRenderer/out.pkl","rb") as f:
#     l = (pickle.load(f))

In [10]:
transform = transforms.Compose([torchvision.transforms.Resize((224,224)),transforms.ToTensor()])
train = MusgraveDataset(transforms = transform)
train_loader = DataLoader(dataset=train, batch_size=10, shuffle=True, num_workers=2)

In [28]:
train[0][0].shape

torch.Size([3, 224, 224])

In [11]:
import torch.optim as optim

# criterion = nn.MSELoss(reduction='sum')
criterion = nn.L1Loss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)

# optimizer = optim.SGD(render.parameters(), lr=0.001, momentum=0.9)

In [17]:
def show(img):
    img = img.detach().cpu().numpy()[0]
    img  = np.transpose(img,(1,2,0))
    print(img.shape)
    cv2_imshow(img)

In [ ]:
# train.l

In [ ]:
# inputs.shape

In [12]:
loss_n = None
for epoch in range(3):
    for i,data in enumerate(train_loader):
        inputs, labels = data[0].cuda(),data[1].cuda()
        labels[:,0]/=12
        labels[:,1]/=16
        labels[:,2]/=4
        labels[:,3]/=3
        # labels/=255
        optimizer.zero_grad()
        
        output = net(inputs)
        
        loss = criterion(output, labels)
        try:
            loss_n+=loss
        except:
            loss_n = loss
        
        if(i%10==9):
            loss_n.backward()
            optimizer.step()
            # print(loss.item())
            print("Epoch:- %2d Loss:- %0.3f i:- %5d".format(epoch,loss_n.item()/10,i))
            # print(output,inputs)
            loss_n = None
        
        

0.4354084014892578
0.26604747772216797
0.2887009859085083
0.2706432342529297
0.2599637985229492
0.2297600507736206
0.21380925178527832
0.20515866279602052
0.23235445022583007
0.22060482501983641
0.21014080047607422
0.22579147815704345
0.1844189167022705
0.19550771713256837
0.19366519451141356
0.20097858905792237
0.1793646454811096
0.20049219131469725
0.2010430097579956
0.19385586977005004
0.1962385058403015
0.18405224084854127
0.18541566133499146
0.19588842391967773
0.18502177000045777
0.1772243857383728
0.1779703378677368
0.18676671981811524
0.18549448251724243
0.18429778814315795
0.17563084363937378
0.19308288097381593
0.1603240728378296
0.17623268365859984
0.180990469455719
0.18837379217147826
0.17961331605911254
0.1650150775909424
0.17637475728988647
0.173515784740448
0.1720563292503357
0.16564981937408446
0.17383790016174316
0.18603036403656006
0.15134215354919434
0.1736816167831421
0.1686345100402832
0.16718153953552245
0.16850292682647705
0.17794013023376465
0.1783687114715576
0

In [13]:
torch.save(net, "/content/drive/My Drive/MusgraveTextureNeuralRenderer/model2.pth")

In [14]:
output

tensor([[ 0.3843,  0.1482,  0.2146,  0.6386],
        [ 0.3903,  0.8230,  0.6964,  0.3965],
        [ 0.4925,  0.7588,  0.9968,  0.7446],
        [ 0.4396,  0.6909,  0.2880,  0.4603],
        [ 0.3473,  0.6302, -0.0038,  0.3761],
        [ 0.4398,  0.7297,  0.6872,  0.3973],
        [ 0.4782,  0.6956,  1.0303, -0.0019],
        [ 0.5404,  0.1816,  0.6542,  1.0488],
        [ 0.4380,  0.9054,  0.2891,  0.4709],
        [ 0.3893,  0.0793,  0.9530,  1.0031]], device='cuda:0',
       grad_fn=<LeakyReluBackward0>)

In [15]:
labels

tensor([[0.3333, 0.1250, 0.2500, 0.0000],
        [0.5833, 0.8750, 0.7500, 0.3333],
        [0.9167, 0.8125, 1.0000, 0.6667],
        [0.7500, 0.7500, 0.2500, 1.0000],
        [0.9167, 0.8125, 0.0000, 0.0000],
        [0.0833, 0.7500, 0.7500, 0.3333],
        [0.8333, 0.7500, 1.0000, 0.0000],
        [0.3333, 0.2500, 0.7500, 1.0000],
        [0.5000, 0.9375, 0.2500, 0.0000],
        [0.5000, 0.0625, 0.7500, 1.0000]], device='cuda:0')

In [ ]:
torch.save(render.state_dict(), "./drive/My Drive/Skin Cancer/RenderResNet.pth")

In [ ]:
torch.save(render.state_dict(), "./drive/My Drive/Skin Cancer/AlexNet.pth")

In [ ]:
def norm(out):
    out-=out.mean()
    out/=out.max()
    return out

In [ ]:
inputs.shape,output.shape

(torch.Size([4, 4]), torch.Size([4, 1000]))

In [ ]:
NetEpochs = 10000
start, stop, step = 7,10,1
Dataset = [x for x in range(1000)]
# Dataset = [0,1]
# Dataset = [0]
done = False
for data in Dataset:
  
  tdata = torch.Tensor([p]).cuda()
  X = model(tdata)
  # print(X)t
  X_CNN = cnn(X)
  # X = X*(255/X.mean())

  X_CNN2 = cnn2(X)
  
  img = process_image(X)
  # print(data,end=" ")
  # print("[ AlexNet "+str(dat[int(torch.argmax(X_CNN).detach().cpu().numpy())]),end="] ")
  # print("[ ResNet "+str(dat[int(torch.argmax(X_CNN2).detach().cpu().numpy())]),end="]")
  
  # cv2_imshow(img)
  cv2.imwrite("dog.jpg",img)
  os.system("python gradcam.py --image-path=dog.jpg --use-cuda")
  cp(data)
  updateJson(data,X_CNN,X_CNN2)
  if((data%10)==0):
    print(data)
  # break
  # imgR = cv2.imread("cam.jpg")
  # cv2_imshow(imgR)
